In [2]:
from utils import Dictionary
tokenizer = Dictionary.load("../data-bin/iwslt15.en-vi//dict.vi.txt")

In [4]:
print([tokenizer[wi] for wi in [2, 7, 5]])

['</s>', 'tôi', '.']


In [61]:
for wi in [    2,     2,    12,   764, 12142,  1110,    12,     2,     2,   764]:
    print(tokenizer[wi])

</s>
</s>
=
Robert
Boul@@
ter
=
</s>
</s>
Robert


In [2]:
import c_fast_model
sos_id, unk_id = tokenizer.index("</s>"), tokenizer.index("<unk>")
lm = c_fast_model.CMultiFastGenerationModel.from_cache("cache_5gram_wt103_bpe/wiki.train.tokens.bpe.repunk.arpa.cache", sos_id, unk_id)

Loaded 119723512 contexts from 120 cache files


In [7]:
data = [[tokenizer.index(it) for it in "= Robert Boulter =".strip().split()]]
word3d_p09, prob3d_p09, ctxlen3d_p09 = lm.batch_dist(data, dist_size=10000, min_dist_prob=0.95, bos=False, require_match_len=True, ctxlens=None)
# word3d_p10, prob3d_p10, ctxlen3d_p10 = lm.batch_dist(data, dist_size=10000, min_dist_prob=1.0, bos=False, require_match_len=True, ctxlens=None)
# word3d_def, prob3d_def, ctxlen3d_def = lm.batch_dist(data, dist_size=10000, bos=False, require_match_len=True, ctxlens=None)

In [46]:
print(sum(10 ** p for p in prob3d_p10[0][2] if p < 0))
print(len(prob3d_p09[0][2]))

0.9858853844938352
10000


In [64]:
str_data = ["</s>", "</s>", "=", "Robert", "Boul@@", "ter", "="]
# data = [tokenizer.index("</s>"), tokenizer.index("</s>"), tokenizer.index("="), tokenizer.index("Boul@@"),  tokenizer.index("ter"), tokenizer.index("=")]
data = [    2,     2,    12,   764, 12142,  1110,    12,     2,     2,   764]
word2d, prob2d, ctxlen2d  = lm.sent_dist(data, dist_size=100, bos=False, require_match_len=True)
print([it[0] for it in ctxlen2d])
print(len(str_data), len(data), len(word2d))
for sm, words in enumerate(word2d):
    print(ctxlen2d[sm][0])
    if sm == len(word2d) - 1:
        break
    flag = data[sm+1] in word2d[sm]
    print(sm, flag)


[1, 1, 2, 2, 1, 2, 2, 1, 1, 2]
7 10 10
1
0 True
1
1 True
2
2 True
2
3 False
1
4 True
2
5 True
2
6 True
1
7 True
1
8 True
2


In [4]:
import torch
DEST="/mnt/task_wrapper/user_output/artifacts/"
train_data = torch.load(DEST+"/results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16//checkpoint_best.pt.train.pd.pt")
valid_data = torch.load(DEST+"/results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16//checkpoint_best.pt.valid.pd.pt")

valid_data = sorted(valid_data, key=lambda x: x[0])

def func(data):
    src = [[tokenizer.index(w) for w in it[2]] for it in data]
    tgt = []
    for it in data:
        for w in it[3]:
            tgt.append(tokenizer.index(w))
    tgt = torch.tensor(tgt, dtype=torch.int)
    tgt = tgt.unsqueeze(-1).repeat(1, 100)
    word3d, prob3d, ctxlen3d = lm.batch_dist(
                src, dist_size=100, bos=False, require_match_len=True, ctxlens=None)
    ngram_ctxlen = []
    for sent in ctxlen3d:
        for it in sent:
            ngram_ctxlen.append(it[0])
    ngram_ctxlen = torch.tensor(ngram_ctxlen, dtype=torch.int)
    ngram_word = []
    for it in word3d:
        for dist in it:
            ngram_word.append(dist)
    ngram_word = torch.tensor(ngram_word, dtype=torch.long)
    gd_match = tgt.eq(ngram_word).sum(dim=-1)
    return ngram_ctxlen.tolist(), gd_match.tolist()

train_info = func(train_data)
valid_info = func(valid_data)

In [13]:
dist_list = []
s = 0
for it in valid_data:
    e = s + len(it[-1])
    pd_score = it[1]
    orig_prob = it[1]
    for i, (j, m, gd) in enumerate(zip(it[-1], valid_info[0][s:e], valid_info[1][s:e])):
        if i < 3:
            continue
        values, indices = torch.topk(orig_prob[:i+1, i], k=min(i, 3))

        indices = (indices.float() * values / values.sum()).sum()
        if torch.isnan(indices):
            continue
        if m == 4:
            dist_list.append(i-indices.item()+1)
    s = e
print(sum(dist_list) / len(dist_list), len(dist_list))


13.609904202664413 13352


In [5]:
import torch
import numpy as np

data_ngram = torch.load("../fairseq-stable/prob.debug.pt")
print(len(data_ngram[0]))

4


In [3]:
import torch
import numpy as np

ctxinfo = torch.load("./wiki.train.tokens.bpe.repunk.ctxinfo.went.pt")

In [28]:
import torch
import numpy as np
import random

random.seed(404)

data_file_list = [
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16/prob.xfm.valid.pt",
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.1/prob.xfm.valid.0.1.pt",
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.1/prob.xfm.valid.0.2.pt",
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.4/prob.xfm.valid.0.4.pt",
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.6/prob.xfm.valid.0.6.pt"
]


data = [
    torch.load(fname) for fname in data_file_list
]


# data = [
#     sorted(torch.load(fname), key=lambda x: x[0]) for fname in data_file_list
# ]

aux_data = data[0]
random.shuffle(aux_data)
data_num = len(aux_data)

# ctx_gd_info = dict()
# ctx_alpha_info = dict()

pos_idx = []
sorted_data = sorted(aux_data, key=lambda x: x[0])
pos = 0
for i in range(data_num):
    tmp_idx = []
    for j in range(len(sorted_data[i][2])):
        tmp_idx.append(pos)
        pos += 1
    pos_idx.append(tmp_idx)


max_ctxlen = 3
my_ctx2ppl = []
n = 0
pos=0
for i in range(data_num):
    gd = aux_data[i][2]
    ctxlens = aux_data[i][3]
    src = aux_data[i][4]
    prob = aux_data[i][1]
    
    for j in range(len(gd)):
        ctx_key = tuple(src[j-max_ctxlen+1:j+1])
        ctx_key_len = len(ctx_key) - 1
        if ctx_key in ctxinfo[ctx_key_len]:
            info = ctxinfo[ctx_key_len][ctx_key]
            if info[0] == 1:
                n+=1
            my_ctx2ppl.append((info[-1], prob[j][1], pos_idx[aux_data[i][0]][j]))
        pos += 1

        # if ctx_key in ctx_gd_info:
        #     ctx_gd_info[ctx_key].append(gd[j])
        # else:
        #     ctx_gd_info[ctx_key] = [gd[j]]
        # max_prob = -float("inf")
        # max_k = 0
        # for k in range(2):
        #     prob = data[k][i][1][j][1]
        #     if max_prob < prob:
        #         max_prob = prob
        #         max_k = k
        # ctx_alpha_info[ctx_key] = 0.2 * max_k

ctx2ppl = sorted(my_ctx2ppl, key=lambda x: x[0])
bucks = []
ppl_res = []
freq_res = []
pos_res = []

for freq, prob, pos in ctx2ppl:
    if len(bucks) >= 10000:
        mean_freq = np.mean([it[0] for it in bucks])
        ppl = np.exp(-sum([it[1] for it in bucks]) / len(bucks))
        mean_pos = np.mean([it[-1] for it in bucks])
        freq_res.append(mean_freq)
        ppl_res.append(ppl)
        pos_res.append(mean_pos)
        bucks = []
    bucks.append((freq, prob, pos))

# print(len(bucks))
# if bucks:
#     mean_freq = np.mean([it[0] for it in bucks])
#     ppl = np.exp(-sum([it[1] for it in bucks]) / len(bucks))
#     freq_res.append(mean_freq)
#     ppl_res.append(ppl)
#     bucks = []

print(freq_res)
print(["{:.2f}".format(it) for it in ppl_res])
print(pos_res)
print(len(ctx2ppl))


FileNotFoundError: [Errno 2] No such file or directory: '../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16/prob.xfm.valid.pt'

In [1]:
"""
Pick the best prob in different systems
"""

import torch
import numpy as np
import random
from collections import Counter
random.seed(404)


def group_ctxinfo(word, prob, ctxlen):
    group_dict = {
        0: [], 1: [], 2: [], 3: [], 4: []
    }
    for i, l in enumerate(ctxlen):
        if prob[i] > 0:
            break
        group_dict[l].append(i)
    ret_dict = dict()
    for m in range(5):
        num = len(group_dict[m])
        if num > 0:
            ctxprob = sum([10 ** prob[it] for it in group_dict[m]])
        else:
            ctxprob = 0
        ret_dict[m] = [num, ctxprob]
    return ret_dict


data_file_list = [
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16/prob.xfm.test.ctxwin1536.pt",
    # "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.seed404/prob.xfm.valid.ctxwin1536.pt",
    # "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.seed123/prob.xfm.valid.ctxwin1536.pt",
    # "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.seed10086/prob.xfm.valid.ctxwin1536.pt",
    # "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.1/prob.xfm.test.0.1.ctxwin1536.pt",
    # "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.2/prob.xfm.valid.0.2.ctxwin1536.pt",
    "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.4/prob.xfm.test.0.4.ctxwin1536.pt",
    # "../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.6/prob.xfm.valid.0.6.ctxwin1536.pt"
]

cnt = 0
data = [
    sorted(torch.load(fname), key=lambda x: x[0]) for fname in data_file_list
]
ex_num = len(data[0])
sys_num = len(data)

prob_res = []
sys_res = []

word_num = 0
prev_src_tokens = None
alpha_pred_data = [[] for _ in range(sys_num)]

gd_pred_test = torch.load("gd_pred_test.dat")

gd_pred_data = []
gd_in_data = []
gd_out_data = []
global_index = 0
for i in range(ex_num):
    gd = data[0][i][2]
    prob_ngram = data[0][i][1]
    ctxlen = data[0][i][3]
    src_token_sid = 0
    src_tokens = data[0][i][4]
    if prev_src_tokens is not None:
        src_token_sid = len(prev_src_tokens)
        src_tokens = prev_src_tokens + data[0][i][4]
    prev_src_tokens = data[0][i][4]

    for j in range(len(gd)):
        ctx = []
        for m in range(4):
            ctx.append(src_tokens[src_token_sid+j-m])
        ctx = ctx[::-1]
        ctx_key = tuple(ctx)
        
        word2d, prob2d, ctxlen2d  = lm.sent_dist(ctx, dist_size=100, min_dist_prob=1.0, bos=False, require_match_len=True)
        word_match, prob_match, ctxlen_match = word2d[-1], prob2d[-1], ctxlen2d[-1]
        grouped_ctx_feat = group_ctxinfo(word_match, prob_match, ctxlen_match)
        assert ctxlen2d[-1][0] == ctxlen[j]
        ctx_feat = [global_index]
        # info = list(ctxinfo[len(ctx_key) - 1][ctx_key])
        # ctx_feat = [global_index, len(ctx_key), info[0], info[1], info[2]]
        for m in range(5):
            if grouped_ctx_feat[m][0] > 0:
                if m > 0:
                    cur_ctx_key = ctx_key[4-m:]
                    cur_ctx_key_len = len(cur_ctx_key)-1
                    info = list(ctxinfo[cur_ctx_key_len][cur_ctx_key])
                else:
                    # info = [114536634, 33339, 10.710820128231383]
                    info = [1, 1, 1]
            else:
                info = [0, 0, 0]
            ctx_feat += grouped_ctx_feat[m] + info
        # assert ctx_key in ctxinfo[ctx_key_len]
        
        max_prob = 0
        max_index = 0
        min_prob = 999
        min_index = 0
        if not data[0][i][1][j][0].endswith("@@"):
            word_num += 1
        # if gd[j] == 1:
        if gd_pred_test[global_index] == gd[j]:
            cnt += 1
        if gd_pred_test[global_index] == 1:
            p = np.exp(data[1][i][1][j][1])
            prob_res.append(p)
            ctx_feat.append(1)
            gd_in_data.append(ctx_feat)
            # gd_in_data.append((global_index, len(ctx_key), info[0], info[1], info[2], 1))
            # in_gd_ctxset.append(ctx)
        else:
            p = np.exp(data[0][i][1][j][1])
            prob_res.append(p)
            ctx_feat.append(0)
            gd_out_data.append(ctx_feat)
            # gd_out_data.append((global_index, len(ctx_key), info[0], info[1], info[2], 0))
        
            # out_gd_ctxset.append(ctx)
        for k in range(sys_num):
            p = np.exp(data[k][i][1][j][1])
            if p > max_prob:
                max_prob = p
                max_index = k
            if p < min_prob:
                min_prob = p
                min_index = k
        ctx_feat.append(max_index)
        alpha_pred_data[max_index].append(ctx_feat)
        prob_res.append(max_prob)
        sys_res.append(max_index)
        global_index += 1
        # if max_prob - min_prob < 0.1:
        #     continue
        # alpha_pred_data.append((global_index, len(ctx_key), info[0], info[1], info[2], max_index))
        
    


print("{:.2f}".format(np.exp(-sum(np.log(prob_res)) / len(prob_res))), len(prob_res))
print("{:.2f}".format(np.exp(-sum(np.log(prob_res)) / word_num), len(prob_res)), word_num)
print(cnt / len(prob_res))
torch.save(alpha_pred_data, "alpha_valid_pred_pro_data_alpha0.4.dat")
# torch.save([gd_in_data, gd_out_data], "gd_valid_pred_pro_data_alpha0.4.dat")
# print(len(gd_in_data), len(gd_out_data))
print(Counter(sys_res))


NameError: name 'lm' is not defined

[4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(4, 1, -1.4426951595367387e-09)


In [20]:
print(data[0][1][3][0])

2


In [18]:
['8.10', '7.79', '6.54', '3.32', '15.28', '13.14', '13.54', '15.28', '18.87', '18.91', '21.43', '25.21', '28.81', '30.80', '38.89', '60.76']
['7.69', '7.42', '6.19', '3.18', '14.64', '12.72', '12.91', '14.56', '18.31', '18.76', '20.89', '24.02', '27.84', '30.17', '39.08', '60.40']

with open("./ctx_gd_info_t0.5.txt", 'w') as fout:
    n = 1
    for ctx, gd_list in ctx_gd_info.items():
        if n == 103:
            print(ctx, [tokenizer[wi] for wi in ctx], gd_list)
        n  += 1
        val = sum(gd_list) / len(gd_list)
        ctx_key_len = len(ctx)-1
        info = ctxinfo[ctx_key_len][ctx]
        # fout.write("{}\t{}\t{:.1f}\n".format(info[0], info[1], 0 if val <= 0.5 else 1))


(5, 81) [',', 'only'] [0, 0, 1, 1, 1, 0]


In [27]:
# import torch
# import numpy as np

# data_ngram = torch.load("../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.2/prob.5gram.test.0.2.pt")
# data_gpt = torch.load("../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16/prob.xfm.test.0.2.pt")

# data_ngram = sorted(data_ngram, key=lambda x: x[0])
# data_gpt = sorted(data_gpt, key=lambda x: x[0])

# score_list = []
# n = 0

# print(data_ngram[0][1][:10])
# print(data_ngram[0][2][:10])
# print(data_ngram[0][3][:10])


# in_gd_ctxset = []
# out_gd_ctxset = []
# for i in range(len(data_gpt)):
#     assert data_ngram[i][0] == data_gpt[i][0]
#     gd = data_ngram[i][2]
#     ctxlens = data_ngram[i][3]
#     gpt_gd = data_gpt[i][2]
#     prob_gpt = data_gpt[i][1]
#     prob_ngram = data_ngram[i][1]
#     assert len(prob_gpt) == len(prob_ngram) and len(prob_ngram) == len(gd)
#     for a, b in zip(gd, gpt_gd):
#         if a != b:
#             raise Exception
#     for j in range(len(gd)):
#         ctx = []
#         for k in range(ctxlens[j]):
#             ctx.append(prob_ngram[j-k][0])
#         ctx = ctx[::-1]
#         if not prob_ngram[j][0].endswith("@@"):
#             n+=1
#         # score_list.append(prob_gpt[j][1])
#         if gd[j] == 1:
#             score_list.append(prob_ngram[j][1])
#             in_gd_ctxset.append(ctx)
#         else:
#             score_list.append(prob_gpt[j][1])
#             out_gd_ctxset.append(ctx)
 
# print(np.exp(-sum(score_list) / len(score_list)), len(score_list))
# print(np.exp(-sum(score_list) / n), len(score_list))

In [28]:
# ctxinfo = torch.load("./wiki.train.tokens.bpe.repunk.ctxinfo.pt")
# in_gd_ctxinfo = dict()
# fuck = []
# for ctx in in_gd_ctxset:
#     if ctx[0] == "</s>":
#         ctx[0] = "<s>"
#     key = tuple([tokenizer.index(it) for it in ctx])
#     clen = len(key)-1
#     if key not in ctxinfo[clen]:
#         fuck.append(ctx)
#         continue
#     if clen in in_gd_ctxinfo:
#         in_gd_ctxinfo[clen].append(ctxinfo[clen][key])
#     else:
#         in_gd_ctxinfo[clen] = [ctxinfo[clen][key]]

# for k, v in in_gd_ctxinfo.items():
#     print(k)
#     print("freq: ", sum([it[0] for it in v]) / len(v))
#     print("#cands: ", sum([it[1] for it in v]) / len(v))

# print(len(in_gd_ctxset))
# print(len(fuck))

    

0
freq:  1251567.1552357625
#cands:  11355.402173913044
1
freq:  36386.894782874886
#cands:  1447.7589758166791
2
freq:  3336.9350377031333
#cands:  213.04534457034558
3
freq:  166.42082744928064
#cands:  35.14234979935963
199497
19580


In [18]:
# tokenizer[0]

'<s>'

In [28]:
# pd_out = torch.load("../results/xfm.base.drop0.3.warm4k.wt103.bpe.fp16.5gram.0.2/checkpoint_best.pt.valid.pd.pt")


In [66]:
# dist_list = []
# max_dist = 0
# index = -1
# pd_matrix = pd_out[0][1]
# for i, j in enumerate(pd_out[0][-1]):
#     if pd_matrix[j, i] < 1e-2:
#         continue
#     dist_list.append(i-j+1)
#     if dist_list[-1] > max_dist:
#         max_dist = dist_list[-1]
#         index = i

# # pd_matrix = pd_out[0][1]
# # print(pd_matrix[:100, 98])
# print(pd_out[0][2][305:795])
# print(pd_out[0][3][305:795])
# print(max_dist, index, pd_out[0][-1][index], pd_out[0][-1][index], pd_out[0][2][pd_out[0][-1][index]], pd_out[0][3][index])
# print(sum(dist_list) / len(dist_list))
# # print(sorted(dist_list)[50])

('Ky@@', 'la', '.', 'I', "'m", 'very', 'sorry', 'to', 'have', 'to', 'let', 'her', 'go', '.', '"', '</s>', '</s>', '=', '=', '=', 'Race', 'and', 'charity', 'work', '=', '=', '=', '</s>', '</s>', 'Throughout', 'her', 'career', ',', 'Ay@@', 'ola', 'has', 'been', 'outspoken', 'on', 'the', 'subject', 'of', 'racial', 'discrimination', 'in', 'the', 'entertainment', 'industry', '.', 'Describing', 'her', 'motivation', ',', 'she', 'states', ':', '"', 'I', 'am', 'not', 'an', 'over@@', 'tly', 'political', 'person', '.', 'I', 'just', 'want', 'fair@@', 'ness', '"', '.', 'Ay@@', 'ola', 'believes', 'that', 'black', 'actors', 'receive', 'less', 'recognition', 'than', 'their', 'white', 'counterparts', ',', 'explaining', ';', '"', 'If', 'you', 'get', 'a', 'show', 'with', 'six', 'stars', 'and', 'one', 'is', 'black', 'you', 'are', 'more', 'likely', 'to', 'see', 'interviews', 'with', 'the', 'five', 'white', 'actors', '.', '[', '...', ']', 'They', 'are', 'not', 'being', 'sold', 'as', 'a', 'reason', 'to', 'wa